# Celebrity Identifier Using Gemini 2.0 Flash

## Introduction

Here, I leverage Gemini 2.0 Flash accessed via the `google.generativeai` library to identify celebrities in images uploaded by the user. It explores prompt engineering for effective interaction with the LLM and structured output for displaying the identified celebrity's information.

## Initial Setup


In [11]:
import google.generativeai as genai
from PIL import Image
import os
from IPython.display import display, Markdown
from dotenv import load_dotenv
import ipywidgets as widgets
from IPython.display import display
import io
from utils import sanitize_filename, generate_unique_filename

## API Setup & Model Initialization

Initialize the Gemini 2.0 Flash model.  We choose the 'gemini-1.5-flash' model.

In [10]:
load_dotenv()

# Get the API key from the environment variables
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

if GOOGLE_API_KEY is None:
    print("Error: GOOGLE_API_KEY not found in environment variables.  Please set it in a .env file.")
    exit()  # Exit if the API key is not found

# Configure the Gemini API
genai.configure(api_key=GOOGLE_API_KEY)


model = genai.GenerativeModel('gemini-1.5-flash')


#creating the img_folder variable to reference the uploaded image folder.
IMG_FOLDER = "static/img"
if not os.path.exists(IMG_FOLDER):
    os.makedirs(IMG_FOLDER)
    print(f"Created '{IMG_FOLDER}' folder.")

## Prompt Engineering and Celebrity Identification Function

This function takes the image path as input, constructs a prompt for the Gemini model, sends the image and prompt to the model, and returns the model's response.  The prompt is crucial for guiding the model to provide the desired output: the name of the celebrity in the image and some relevant information.

In [12]:
def identify_celebrity(image_path):
    """Identifies the celebrity in the image using Gemini 2.0 Flash.

    Args:
        image_path: The path to the image file.

    Returns:
        A string containing the model's response, or None if there was an error.
    """
    try:
        img = Image.open(image_path)
        # resize the image to a maximum of 1024x1024 to prevent issues with large images
        img.thumbnail((1024, 1024))

        prompt_text = """
        You are an expert celebrity identifier.  Analyze the image provided and identify the celebrity in the image.
        Provide the celebrity's name, a brief biography (2-3 sentences), a table highlighting details of their most popular works, and three interesting facts about them.
        Format your response as follows:

        **Celebrity Name:** [Celebrity's Full Name]

        **Biography:**
        [2-3 sentence biography]
        
        **Work History:**
        [Table with details of their most popular works]

        **Interesting Facts:**
        1.  [Fact 1]
        2.  [Fact 2]
        3.  [Fact 3]
        """

        response = model.generate_content([prompt_text, img])
        return response.text
    except Exception as e:
        print(f"Error processing image: {e}")
        return None


## Image Upload Function

In [14]:
def on_upload_success(change):
    global image_path  # Declare image_path as global
    if len(change['new']) > 0:
        uploaded_file = change['new'][0]  # Get the file information

        filename = uploaded_file['name']  # Get the filename
        content = uploaded_file['content']  # Get the file content (bytes)

        # Sanitize filename (important for security)
        filename = sanitize_filename(filename)

        # Generate unique filename (prevent collisions)
        filename = generate_unique_filename(filename)

        # Construct file path
        image_path = os.path.join(IMG_FOLDER, filename)

        # Save the image to the static folder
        try:
            with open(image_path, 'wb') as f:
                f.write(content)
            print(f"Image '{filename}' saved to '{IMG_FOLDER}'")

            # Now proceed with celebrity identification
            celebrity_info = identify_celebrity(image_path)

            # Display the results
            if celebrity_info:
                print("Celebrity Identification Results:")
                display(Markdown(celebrity_info))  # Use Markdown for better formatting
            else:
                print("Could not identify the celebrity in the image.")

        except Exception as e:
            print(f"Error saving image: {e}")
            image_path = None
    else:
        print("No file uploaded.")
        image_path = None


## Main Execution Block

This part of the notebook orchestrates the image upload, celebrity identification, and displays the result in a structured format.

In [15]:
# 1. Upload the image and get the file path.
uploader = widgets.FileUpload(
    accept='image/*',  # Allow only image files
    multiple=False  # Allow only one file to be uploaded
)

image_path = None  # Initialize image_path outside the callback

uploader.observe(on_upload_success, names=['value'])
display(uploader)

FileUpload(value=(), accept='image/*', description='Upload')

Image '8eec1981-3f86-422b-a02e-c8ab6e571f74.jpeg' saved to 'static/img'
Celebrity Identification Results:


**Celebrity Name:** Sofia Vergara

**Biography:**

Sofia Vergara is a Colombian-American actress, best known for her role as Gloria Delgado-Pritchett on the ABC sitcom *Modern Family*.  Her comedic timing and vibrant personality have made her one of television's highest-paid actresses, and she's also a successful businesswoman with various endorsements and ventures.  Beyond acting, she has a significant presence in the media and entertainment industry as a whole.


**Work History:**

| Work Title             | Type          | Year      | Notes                                         |
|------------------------|----------------|------------|---------------------------------------------|
| Modern Family          | Television Series | 2009-2020 | Breakout role; earned numerous awards and nominations |
| Chef                       | Film          | 2012      | Notable film role                               |
| Hot Pursuit              | Film          | 2015      | Leading role alongside Reese Witherspoon         |
|  Smurfs Movie Franchise| Film          | 2011-2017| Voice acting role                            |


**Interesting Facts:**

1.  Sofia Vergara was discovered while walking on a beach in Barranquilla, Colombia, leading to her early modeling and television career.
2.  She is a successful businesswoman, owning her own production company and having numerous successful endorsements.
3.  Before achieving international fame, Vergara was a popular TV host in Colombia.


<h3>Author:</h2>

<a href="https://www.linkedin.com/in/mnnamchi/">Martins Nnamchi</a>
